## Introducción

En este notebook se procederá ha realizar un filtrado de los datos. Algunos de los ficheros utilizados contienen millones de registros, por lo que un filtrado inicial ayudará a esclarecer qué tipo de datos tenemos y, además, contribuirá a que el análisis visual no consuma demasiado tiempo de cómputo.

Para realizar este filtrado, primero se ha hecho uso de la librería `Pandas Profiling`, la cual permite obtener un análisis completo de los sets de datos, indicando entre otros aspectos variables conflictivas en cuanto a cardinalidad, uniformidad, distribución, etc. Tras mostrar el análisis con Profiling, se ha procedido ha realizar un estudio para determinar qué columnas o variables eliminar de nuestros sets de datos, tanto si son variables conflictivas como si son variables que no consideramos útiles a la hora de obtener patrones.

Cabe destacar que en ciertos casos se ha llevado a cabo un proceso inverso, es decir, primero el filtrado y luego el Profiling, puesto que el Profiling consume demasiada RAM y tiempo de cómputo para sets de datos con un elevado número de instancias.

*Nota: Inicialmente se realizó un filtrado de los csv correspondientes a las reviews y los usuarios. Sin embargo, finalmente estos archivos no han sido necesarios para nuestro propósito, pero se dejarán como evidencia del filtrado inicial.*

Ejecutar primero esta instruccion y reiniciar el entorno de ejecucion para habilitar Pandas Profiling

In [ ]:
!pip install pandas-profiling==2.7.1

## Para montar el drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/AnalisisDeDatos/PracticaFinal/

/content/drive/MyDrive/AnalisisDeDatos/PracticaFinal


### Imports y utilidades

In [ ]:
import pandas as pd
import IPython
from pandas_profiling import ProfileReport

In [ ]:
import gc
gc.collect()

117580

## Filtrado inicial

### Negocios

En este apartado, se va a filtrar y guardar los datos referentes a los negocios en 
"business". Despues, se generará un profile report de los negocios para 
posteriormente realizar un análisis en profundidad de los atributos que contienen los negocios. Ese 'profile report' se guardará en un fichero HTML.

In [ ]:
business = pd.read_csv('data/business/business_data.csv', sep=",")

NameError: ignored

In [ ]:
business

In [ ]:
prof = ProfileReport(business)
prof.to_file(output_file='analysis/output_business.html')

In [ ]:
IPython.display.HTML(filename='analysis/output_business.html') 

Si se ejecuta el código superior se puede ver un análisis estadístico completo de los datos referentes a los **negocios**.

A continuación, se procede a realizar un limpiado de los datos, de manera que se tendrán que eliminar aquellas columnas correspondientes a los negocios que no proporcionen ninguna información útil para la tarea de la predicción.
Se ha eliminado `name`, porque es identificativo de cada negocio y ya se dispone el `id`, los atributos, que no aportan información, al igual que la dirección y el estado, ya que se mantiene su `zipcode`. Por último, se eliminan las coordenadas del negocio porque ya se dispone de esa información y se eliminan las horas que no aportan información. Este filtrado se guarda posteriormente en un archivo `.csv`.

In [ ]:
business_filtered = business.drop(columns=['name','attributes', 'address', 'state', 'lat', 'long', 'hours'])


Si se observa el reporte generado por Pandas Profiling, puede verse que algunas variables tienen valores vacíos o NaN (Not a Number). Por tanto, se ha decidido eliminar aquellas filas cuyas variables cumplan esta condición. En concreto, las variables `zipcode` y `categories` presentan valores vacíos.

In [ ]:
business_filtered.dropna(subset = ["zipcode", "categories"], inplace=True)

In [ ]:
business_filtered

In [ ]:
business_filtered.to_csv("data/business/business_filtered.csv", index=False)

### Opiniones

En este apartado se va a filtrar y guardar los datos referentes a las opiniones de los clientes o usuarios. Para tratar las opiniones, como disponemos de la fecha, solo se van a tratar aquellas opiniones posteriores al año 2018. Después, se realizará un profile report para analizar en profundidad la importancia de los atributos que contiene cada una de las opiniones.

In [ ]:
review = pd.read_csv('data/review/review_data.csv', sep=",")

In [ ]:
review

In [ ]:
review['date'] = pd.to_datetime(review.date)
review['date'] = review['date'].dt.strftime('%Y')
review = review[(review['date'] >= '2018')]
review = review.drop(columns=['date'])

In [ ]:
review

In [ ]:
prof = ProfileReport(review)
prof.to_file(output_file='analysis/output_review.html')

In [ ]:
# Con esta linea mostramos el output del profiling sin tener que abrir el HTML.
IPython.display.HTML(filename='analysis/output_review.html')

De la misma manera que en el apartado de negocios, se va a proceder a realizar un filtrado y limpiado de los datos para eliminar aquellas columnas correspondientes a las opiniones que no aporten información o aporten ruido. Hemos eliminado las columnas "useful, "cool y "fun" que son atributos numéricos que no aportan información para la tarea de la predición. La descripción, a pesar de ser un valor útil, será eliminada también puesto que ya disponemos de otros atributos que son más utiles para calificar o valorar un negocio. Posteriormente, el filtrado se guarda en un fichero csv.

In [ ]:
review_filtered = review.drop(columns=['useful', 'cool', 'fun', 'description'])

In [ ]:
review_filtered

In [ ]:
review_filtered.to_csv('data/review/review_filtered.csv', index=False)

### Usuarios

Al igual que para los negocios y las opiniones, se va a realizar un preprocesado de los datos referentes a los usuarios. Aquí, se va a realizar un profile report para poder analizar la importancia de los atributos y sacar conclusiones para el filtrado. 

In [ ]:
user = pd.read_csv('data/user/user_data.csv', sep=",")

In [ ]:
user

En las siguientes líneas de código se podrá ver el reporte proporcionado por Pandas Profiling. Sin embargo, hemos creído conveniente realizar un pequeño pero importante filtrado previo. En concreto, no nos interesan aquellos usuarios que no han realizado ninguna opinions sobre ningún negocio, por lo que hemos decidido eliminarlos.

In [ ]:
user_filtered = user[(user['num_reviews'] > 0)]

In [ ]:
prof = ProfileReport(user_filtered)
prof.to_file(output_file='analysis/output_user.html')

In [ ]:
IPython.display.HTML(filename='analysis/output_user.html')

Si observamos el reporte proporcionado por Pandas Profiling, podremos ver que hay numerosas variables conflictivas en este set de datos. Vamos a analizarlas una por una:



*   `name`: Esta variable correspondiente al nombre del usuario posee una gran cardinalidad puesto que cada usuario tendrá un nombre asociado. Además, es una variable que no proporcionará información ni para un análisis visual ni para el posterior proceso de Machine Learning.

*   `user_since`: Se trata de una variable con formato fecha que expresa el momento en que el usuario se registró. Si bien podría ser útil segmentar por usuarios que llevan mucho tiempo dando opiniones, creemos que se perdería mucha información sobre nuevos usuarios, por lo que se eliminará del set de datos.

*   `useful`: Esta variable es un contador de cuántas veces el usuario ha sido valorado como útil. Es una variable que puede ser valiosa, el problema es que contiene una enorme cantidad de ceros, en concreto un 26% de las instancias contienen un cero.
*   `fun`: Al igual que `useful` esta variable contiene una gran cantidad de ceros, en concreto casi 53%. No será de gran utilidad así que será eliminada
*   `cool`: Posee un 50% de ceros, así que se elimina.
*   `expert`: Esta variable proporciona los años en que el usuario estuvo activo dando opiniones. Sin embargo, esta variable posee un formato que no está soportado, por lo que será eliminada.
*   `friends`: En el dataset, hay en total 689048 usuarios que no tienen ningún amigo registrado. Por tanto, esta variable no es útil para el análisis.
*   `likes`: Los likes son una serie de variables que representan los 'me gusta' de distintos tipos dados al usuario. La mayor parte de lo valores que toman estas variables es cero, por lo que no se tendrán en cuenta.












A continuación, se van a eliminar aquellas columnas que no aportan información. Como son bastantes, en lugar de comentar las que se van a eliminar, se comentarán aquellos atributos  que se van a quedar y que por tanto vamos a utilizar en la tarea de la predicción. 

En primer lugar, nos quedaremos con el identificador del usuario, ya que así se puede hacer una correspondencia entre el usuario y la review que ha realizado sobre los negocios. 

En segundo lugar, queremos ponderar la validez de un usuario, y para ello utilizamos el número de opiniones que ha realizado y el número de seguidores que tiene. Esto se debe a que la opinión de un usuario que tiene muchos seguidores y muchas opiniones realizadas deberán tener una ponderación mayor que las de un usuario que tiene pocos seguidores o no ha realizado ninguna valoración. 

Por último, se utiliza el rating del usuario de la mimsa manera que los atributos anteriores y valorar al usuario. 



In [ ]:
# Filtrado basico
user_filtered = user_filtered[['user_id', 'num_reviews', 'followers', 'average_rating']]

No se han encontrado evidencias de que el dataset contenga valores vacíos, por lo que no tendremos que hacer ningún filtrado al respecto.

In [ ]:
user_filtered

In [ ]:
user_filtered.to_csv('data/user/user_filtered.csv', index=False)